<a href="https://colab.research.google.com/github/yisiszhang/AdvancedPython/blob/main/Precept3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Precept 3
Neural network exercise

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

You can try another dataset from the open database, or you can bring your own data to train a neural network classifier.

In [2]:
from torchvision.datasets import FashionMNIST
random_seed = 1
torch.manual_seed(random_seed)
# load training and test sets of the MNIST dataset
data_train = FashionMNIST("./data", train=True, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ]))
data_test = FashionMNIST("./data", train=False, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ]))
print(data_train)
print(data_test)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [39]:
# if you bring your own data, you can also convert it to tensor format
# e.g. you load your data into Pandas dataframe
import pandas as pd
digits = pd.read_csv('sample_data/mnist_train_small.csv',header=None).values
labels = digits[:,0] # the first column of the data are the labels
images = digits[:,1:].reshape(len(digits), 28, 28) # convert vectorized image back to original 2d format
#images = images/images.max() # scale to 0 and 1

train = torch.utils.data.TensorDataset(torch.Tensor(images), torch.Tensor(labels))
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=10, shuffle=True)
print(train_loader)

In [40]:
# set up hyperparameters
n_epochs = 10
batch_size_train = 100
batch_size_test = len(data_test) # use the entire test data
learning_rate = 0.001

train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size_test, shuffle=False)
print(len(train_loader))

600


In [41]:
# define a neural network with 1 hidden layer
class NeuralNet(nn.Module):
  # modify the following such that the class has 3 input variables: number of
  # inputs, number of neurons in the hidden layer, and number of outputs
  def __init__(self):
    super(NeuralNet, self).__init__()
    # modify below such that the first layer has n input and m output
    self.layer1 = 0
    # modify below such that the second layer has m input and o output
    self.layer2 = 0
    # modify below to determine an activation function
    self.activation = 0
  def forward(self, x):
    x = torch.flatten(x, start_dim=1)
    # complete this method such that the forward pass does linear computation
    # of layer 1 and activation function, then linear computation of layer 2
    o = 0
    return o

In [44]:
# modify below such that the input and output sizes match the data, and you can define the number of neurons in hidden layer 
input_size = 0
hidden_size = 0
num_output = 0

# modify the input of the class instance
model = NeuralNet()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
nbatch = len(train_loader)
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs)]

for epoch in range(n_epochs):
  for i, (images, labels) in enumerate(train_loader):
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
      print('Train Epoch: {} [{} / {} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, i * len(labels), len(train_loader.dataset), 100. * i/len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(i*train_loader.batch_size + epoch*len(train_loader.dataset))

  test_loss = 0
  correct = 0
  nsample = 0
  with torch.no_grad():
    for images, labels in test_loader:
      outputs = model(images)
      loss = criterion(outputs, labels)
      test_loss += loss.item()
      pred = outputs.argmax(axis=1)
      correct += (pred == labels).sum().item()
      nsample += len(labels)
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  acc = 100.0 * correct/nsample
  print(f'Test accuracy = {acc} %')